##### Data collapse (improved)

In [168]:
using JLD2
N = 7
v = 1
test_values = zeros(ComplexF64,(2*N-1)^2)

l = length(test_values)

distx = 0.0007## distance from centre in real

disty = 0.0007# distance from centre in imaginary

cent_im = 0.0600im

cent_r = 0.0780

## snake like structure of test_values will allow for faster convergence when recycling ψ (because you don't jump the entire distx after the loop)
for i in 1:1:(2*N-1)
    if div(i,2) == 1
        for j in 1:1:(2*N-1)
            if i <N+1
                if j<N+1
                    test_values[i+(j-1)*(2*N-1)] =  (LinRange(-distx,0.00,N)[i])  .+cent_r+ 1im*LinRange(-disty,0.00,N)[j] .+cent_im
                else 
                    test_values[i+(j-1)*(2*N-1)] =  (LinRange(-distx,0.00,N)[i])  .+cent_r + 1im*LinRange(disty/(N-1),disty+disty/(N-1),N)[j-N] .+cent_im
                end
            else
                if j<N+1
                    test_values[i+(j-1)*(2*N-1)] =  (LinRange(distx/(N-1),distx+distx/(N-1),N)[i-N]) .+cent_r  + 1im*LinRange(-disty,0.00,N)[j]  .+cent_im
                else
                    test_values[i+(j-1)*(2*N-1)] =  (LinRange(distx/(N-1),distx+distx/(N-1),N)[i-N])  .+cent_r+ 1im*LinRange(disty/(N-1),disty+disty/(N-1),N)[j-N].+cent_im
                end
            end 
        end
    else
        for j in (2*N-1):-1:1
            if i <N+1
                if j<N+1
                    test_values[i+(j-1)*(2*N-1)] =  (LinRange(-distx,0.00,N)[i]) .+cent_r+ 1im*LinRange(-disty,0.00,N)[j] .+cent_im
                else 
                    test_values[i+(j-1)*(2*N-1)] =  (LinRange(-distx,0.00,N)[i])  .+cent_r + 1im*LinRange(disty/(N-1),disty+disty/(N-1),N)[j-N] .+cent_im
                end
            else
                if j<N+1
                    test_values[i+(j-1)*(2*N-1)] =  (LinRange(distx/(N-1),distx+distx/(N-1),N)[i-N]) .+cent_r + 1im*LinRange(-disty,0.00,N)[j] .+cent_im
                else
                    test_values[i+(j-1)*(2*N-1)] =  (LinRange(distx/(N-1),distx+distx/(N-1),N)[i-N])   .+cent_r + 1im*LinRange(disty/(N-1),disty+disty/(N-1),N)[j-N] .+cent_im
                end
            end 
        end
    end
end

Trying data collaps with sigma'

In [169]:
function C_TbarT(h)
    return h^2 - h*(cth)/12

end

function C_T2T2(h)
    return h^2 - h*(cth+2)/12
end

hσ  = (0.133596708540452 - 0.0204636065293973im)/2
hε  = (0.465613165838194 - 0.224494536412444im)/2
hσprime = (1.11067215243258 - 0.17012663080436im)/2

function coef1(hσ,hε,hσprime)
    a = (C_TbarT(hσprime)/C_TbarT(hσ)) * ( -1 + ( (C_TbarT(hε) * C_T2T2(hσ))/( C_T2T2(hσ)*C_TbarT(hε) -C_TbarT(hσ)* C_T2T2(hε)) )) - (C_TbarT(hε) * C_T2T2(hσprime))/( C_T2T2(hσ)*C_TbarT(hε) -C_TbarT(hσ)* C_T2T2(hε))
    return a
end
function coef2(hσ,hε,hσprime)
    a = ( C_T2T2(hσprime)* C_TbarT(hσ) -C_TbarT(hσprime)* C_T2T2(hσ) )/( C_T2T2(hσ)*C_TbarT(hε) -C_TbarT(hσ)* C_T2T2(hε)) 
return a
end

coef2 (generic function with 1 method)

In [170]:
coef2(hσ,hε,hσprime)

-4.230232293086518 - 3.536446908452194im

In [171]:
using PlotlyJS
E_collaps19 = []
E_collaps20 = []
L = 19
for (i,lambda) in enumerate(test_values)
            lambda_txt = round(lambda,digits = 6)
          
            results1 = load_object("Lambda_est_precise/PBC/PBC_D_500/L$L/5EXC0_PBC_L=$L"*"_$lambda_txt"*"_D100.jld2")
            results2 = load_object("Lambda_est_precise/PBC/PBC_D_500/L$L/5EXC1_PBC_L=$L"*"_$lambda_txt"*"_D100.jld2")
        push!(E_collaps19,L*(results2[2][end-1] + coef1(hσ,hε,hσprime)*results1[2][1]+coef2(hσ,hε,hσprime)*results2[2][end]) )
    end

    L = 20
    for (i,lambda) in enumerate(test_values)
        lambda_txt = round(lambda,digits = 6)
        if L ==20 && (lambda_txt == 0.078233 +0.059883im)
            push!(E_collaps20,0)
        else
            results1 = load_object("Lambda_est_precise/PBC/PBC_D_500/L$L/5EXC0_PBC_L=$L"*"_$lambda_txt"*"_D100.jld2")
            results2 = load_object("Lambda_est_precise/PBC/PBC_D_500/L$L/5EXC1_PBC_L=$L"*"_$lambda_txt"*"_D100.jld2")
            push!(E_collaps20,L*(results2[2][end-1] + coef1(hσ,hε,hσprime)*results1[2][1]+coef2(hσ,hε,hσprime)*results2[2][end] ))
        end
    end

In [172]:
using PlotlyJS
z_values = []
    test_alt = []
    for res in 1:1:length(test_values)
        lambda_txt = round(test_values[res],digits = 6)
        if L ==20 && (lambda_txt == 0.078233 +0.059883im)
        else
        push!(z_values,log.(abs.((E_collaps20[res]-E_collaps19[res]))))
        push!(test_alt,test_values[res])
        end
    end
p = PlotlyJS.plot(PlotlyJS.contour(   z=z_values,
         x=real(-im*test_alt),
             y=real(test_alt),fill=true,colorbar=attr(
                 title="log(|gε'|)", # title here
                titleside="top",
                titlefont=attr(
                  size=14,
                  family="Arial, sans-serif"
              )
             )),Layout(title=attr(text = "Data collaps L19-L20 with Eσ Eε Eσ' L19-L20 D = 500",x = 0.5),xaxis_title="Im(λ)",yaxis_title="Re(λ)")
     )
PlotlyJS.savefig(p,"Data_collaps.png")  

"Data_collaps.png"

With L-1ε

In [173]:
function C_T2T2_desc(h)
    return (h+1)^2 - (h+1)*(cth+2)/12+2(h+1)+h^2 - h*(cth)/12
end
function C_TbarT_desc(h)
    return h*(h+1)^2 -(2h+1)*(cth)/24

end
### also previous C_T2T2 terms must be multiplied by 2 since we redfine z= gT2 = g_barT^2 so (gT2 + g_barT^2) = 2z of previous



function coef1(hσ,hε,hσprime)
    a = (C_TbarT_desc(hε)/C_TbarT(hσ)) * ( -1 + ( (C_TbarT(hε) * 2*C_T2T2(hσ))/( 2*C_T2T2(hσ)*C_TbarT(hε) -C_TbarT(hσ)*2* C_T2T2(hε)) )) - (C_TbarT(hε) * C_T2T2_desc(hε))/( C_T2T2(hσ)*C_TbarT(hε) -C_TbarT(hσ)* C_T2T2(hε))
    return a
end
function coef2(hσ,hε,hσprime)
    a = ( C_T2T2_desc(hε)* C_TbarT(hσ) -C_TbarT_desc(hε)* C_T2T2(hσ) )/( C_T2T2(hσ)*C_TbarT(hε) -C_TbarT(hσ)* C_T2T2(hε)) 
return a
end

coef2 (generic function with 1 method)

In [174]:
using PlotlyJS
E_collaps19 = []
E_collaps20 = []
L = 19
for (i,lambda) in enumerate(test_values)
            lambda_txt = round(lambda,digits = 6)
          
            results1 = load_object("Lambda_est_precise/PBC/PBC_D_500/L$L/5EXC0_PBC_L=$L"*"_$lambda_txt"*"_D100.jld2")
            results2 = load_object("Lambda_est_precise/PBC/PBC_D_500/L$L/5EXC1_PBC_L=$L"*"_$lambda_txt"*"_D100.jld2")
        push!(E_collaps19,L*(results2[2][end-1] + coef1(hσ,hε,hσprime)*results1[2][1]+coef2(hσ,hε,hσprime)*results1[2][2]) )
    end

    L = 20
    for (i,lambda) in enumerate(test_values)
        lambda_txt = round(lambda,digits = 6)
        if L ==20 && (lambda_txt == 0.078233 +0.059883im)
            push!(E_collaps20,0)
        else
            results1 = load_object("Lambda_est_precise/PBC/PBC_D_500/L$L/5EXC0_PBC_L=$L"*"_$lambda_txt"*"_D100.jld2")
            results2 = load_object("Lambda_est_precise/PBC/PBC_D_500/L$L/5EXC1_PBC_L=$L"*"_$lambda_txt"*"_D100.jld2")
            push!(E_collaps20,L*(results2[2][end-1] + coef1(hσ,hε,hσprime)*results1[2][1]+coef2(hσ,hε,hσprime)*results1[2][2] ))
        end
    end

In [175]:
using PlotlyJS
z_values = []
    test_alt = []
    for res in 1:1:length(test_values)
        lambda_txt = round(test_values[res],digits = 6)
        if L ==20 && (lambda_txt == 0.078233 +0.059883im)
        else
        push!(z_values,abs((E_collaps20[res]-E_collaps19[res])))
        push!(test_alt,test_values[res])
        end
    end
p = PlotlyJS.plot(PlotlyJS.contour(   z=z_values,
         x=real(-im*test_alt),
             y=real(test_alt),fill=true,colorbar=attr(
                 title="log(|gε'|)", # title here
                titleside="top",
                titlefont=attr(
                  size=14,
                  family="Arial, sans-serif"
              )
             )),Layout(title=attr(text = "Data collaps with Eσ Eε EL-1ε L19-L20 D = 500",x = 0.5),xaxis_title="Im(λ)",yaxis_title="Re(λ)")
     )
PlotlyJS.savefig(p,"Data_collaps2.png")  

"Data_collaps2.png"